In [1]:
import sys

sys.path.append("../")

import torch
import torch.utils as utils

import pytorch_lightning as pl

from torchemlp.groups import SO, O, S, Z
from torchemlp.nn.equivariant import EMLP
from torchemlp.nn.runners import FuncMSERegressionLightning
from torchemlp.nn.utils import Standardize
from torchemlp.datasets import O5Synthetic

In [2]:
TRAINING_SET_SIZE = 5_000
VALIDATION_SET_SIZE = 1_000
TEST_SET_SIZE = 1_000

BATCH_SIZE = 500

N_EPOCHS = 5  # min(int(900_000 / TRAINING_SET_SIZE), 1000)
# N_EPOCHS = 100  # min(int(900_000 / TRAINING_SET_SIZE), 1000)

N_CHANNELS = 384
N_LAYERS = 3

DL_WORKERS = 0

In [3]:
dataset = O5Synthetic(
    TRAINING_SET_SIZE + VALIDATION_SET_SIZE + TEST_SET_SIZE, device="cuda"
)
f"Input type: {dataset.repin(dataset.G)}, output type: {dataset.repout(dataset.G)}"

'Input type: 2V, output type: V⁰'

In [4]:
split_data = utils.data.random_split(
    dataset, [TRAINING_SET_SIZE, VALIDATION_SET_SIZE, TEST_SET_SIZE]
)

train_loader = utils.data.DataLoader(
    split_data[0], batch_size=BATCH_SIZE, num_workers=DL_WORKERS, shuffle=True
)
val_loader = utils.data.DataLoader(
    split_data[1], batch_size=BATCH_SIZE, num_workers=DL_WORKERS
)
test_loader = utils.data.DataLoader(
    split_data[2], batch_size=BATCH_SIZE, num_workers=DL_WORKERS
)

In [5]:
model = Standardize(
    EMLP(dataset.repin, dataset.repout, dataset.G, N_CHANNELS, N_LAYERS), dataset.stats
).cuda()
plmodel = FuncMSERegressionLightning(model)

In [6]:
trainer = pl.Trainer(
    limit_train_batches=BATCH_SIZE, max_epochs=N_EPOCHS, accelerator="gpu"
)
trainer.fit(plmodel, train_loader, val_loader)
trainer.test(plmodel, test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type        | Params
--------------------------------------
0 | model | Standardize | 480 K 
--------------------------------------
480 K     Trainable params
0         Non-trainable params
480 K     Total params
1.923     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/rytse/mambaforge/envs/l2e10/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/rytse/Documents/research/torchemlp/examples/../torchemlp/reps/reps_base.py:1381: UserWarning: operator() sees varying value in profiling, ignoring and this should be handled by GUARD logic (Triggered internally at ../third_party/nvfuser/csrc/parser.cpp:3777.)
  return lazy_proj_jit(
/home/rytse/Documents/research/torchemlp/examples/../torchemlp/reps/reps_base.py:1381: UserWarning: operator() profile_node %119 : int[] = prim::profile_ivalue[profile_failed="varying profile values"](%117)
 does not have profile information (Triggered internally at ../third_party/nvfuser/csrc/gr

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/rytse/mambaforge/envs/l2e10/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    35.857906341552734     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 35.857906341552734}]